---
title: "Tutorial: Preprocess your data with recipes"
execute: 
  enabled: true
---

Prepare data for modeling with modular preprocessing steps.

# Introduction

...

In [1]:
%%capture
import ibis

con = ibis.connect("duckdb://nycflights13.ddb")
con.create_table(
    "flights", ibis.examples.nycflights13_flights.fetch().to_pyarrow(), overwrite=True
)
con.create_table(
    "weather", ibis.examples.nycflights13_weather.fetch().to_pyarrow(), overwrite=True
)

You can now see the example dataset copied over to the database:

In [2]:
con = ibis.connect("duckdb://nycflights13.ddb")
con.list_tables()

['flights', 'weather']

We’ll turn on interactive mode, which partially executes queries to give users a preview of the results. 

In [3]:
ibis.options.interactive = True

In [4]:
flights = con.table("flights")
flights = flights.mutate(
    dep_time=(
        flights.dep_time.lpad(4, "0").substr(0, 2)
        + ":"
        + flights.dep_time.substr(-2, 2)
        + ":00"
    ).try_cast("time"),
    arr_delay=flights.arr_delay.try_cast(int),
    air_time=flights.air_time.try_cast(int),
)
flights

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ time     │ int64          │ string    │ string   │ int64          │ int64     │ string  │ int64  │ string  │ string │ string │ int64    │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 05:17:00 │            515 │ 2         │ 830      │            819 │        11 │ UA      │   1545 │ N14228  │ EWR    │ IAH    │      227 │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 05:33:00 │            529 │ 4         │ 850      │            830 │        20 │ UA      │   1714 │ N24211  │ LGA    │ IAH    │      227 │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 05:42:00 │            540 │ 2         │ 923      │            850 │        33 │ AA      │   1141 │ N619AA  │ JFK    │ MIA    │      160 │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 05:44:00 │            545 │ -1        │ 1004     │           1022 │       -18 │ B6      │    725 │ N804JB  │ JFK    │ BQN    │      183 │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 05:54:00 │            600 │ -6        │ 812      │            837 │       -25 │ DL      │    461 │ N668DN  │ LGA    │ ATL    │      116 │      762 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 05:54:00 │            558 │ -4        │ 740      │            728 │        12 │ UA      │   1696 │ N39463  │ EWR    │ ORD    │      150 │      719 │     5 │     58 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 05:55:00 │            600 │ -5        │ 913      │            854 │        19 │ B6      │    507 │ N516JB  │ EWR    │ FLL    │      158 │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 05:57:00 │            600 │ -3        │ 709      │            723 │       -14 │ EV      │   5708 │ N829AS  │ LGA    │ IAD    │       53 │      229 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 05:57:00 │            600 │ -3        │ 838      │            846 │        -8 │ B6      │     79 │ N593JB  │ JFK    │ MCO    │      140 │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 05:58:00 │            600 │ -2        │ 753      │            745 │         8 │ AA      │    301 │ N3ALAA  │ LGA    │ ORD    │      138 │      733 │     6 │      0 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │         … │ …       │      … │ …       │ …      │ …      │        … │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

In [5]:
weather = con.table("weather")
weather

┏━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ origin ┃ year  ┃ month ┃ day   ┃ hour  ┃ temp   ┃ dewp   ┃ humid  ┃ wind_dir ┃ wind_speed         ┃ wind_gust ┃ precip  ┃ pressure ┃ visib   ┃ time_hour           ┃
┡━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string │ int64 │ int64 │ int64 │ int64 │ string │ string │ string │ string   │ string             │ string    │ float64 │ string   │ float64 │ timestamp(6)        │
├────────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┼──────────┼────────────────────┼───────────┼─────────┼──────────┼─────────┼─────────────────────┤
│ EWR    │  2013 │     1 │     1 │     1 │ 39.02  │ 26.06  │ 59.37  │ 270      │ 10.357019999999999 │ NA        │     0.0 │ 1012     │    10.0 │ 2013-01-01 06:00:00 │
│ EWR    │  2013 │     1 │     1 │     2 │ 39.02  │ 26.96  │ 61.63  │ 250      │ 8.05546            │ NA        │     0.0 │ 1012.3   │    10.0 │ 2013-01-01 07:00:00 │
│ EWR    │  2013 │     1 │     1 │     3 │ 39.02  │ 28.04  │ 64.43  │ 240      │ 11.5078            │ NA        │     0.0 │ 1012.5   │    10.0 │ 2013-01-01 08:00:00 │
│ EWR    │  2013 │     1 │     1 │     4 │ 39.92  │ 28.04  │ 62.21  │ 250      │ 12.658579999999999 │ NA        │     0.0 │ 1012.2   │    10.0 │ 2013-01-01 09:00:00 │
│ EWR    │  2013 │     1 │     1 │     5 │ 39.02  │ 28.04  │ 64.43  │ 260      │ 12.658579999999999 │ NA        │     0.0 │ 1011.9   │    10.0 │ 2013-01-01 10:00:00 │
│ EWR    │  2013 │     1 │     1 │     6 │ 37.94  │ 28.04  │ 67.21  │ 240      │ 11.5078            │ NA        │     0.0 │ 1012.4   │    10.0 │ 2013-01-01 11:00:00 │
│ EWR    │  2013 │     1 │     1 │     7 │ 39.02  │ 28.04  │ 64.43  │ 240      │ 14.960139999999999 │ NA        │     0.0 │ 1012.2   │    10.0 │ 2013-01-01 12:00:00 │
│ EWR    │  2013 │     1 │     1 │     8 │ 39.92  │ 28.04  │ 62.21  │ 250      │ 10.357019999999999 │ NA        │     0.0 │ 1012.2   │    10.0 │ 2013-01-01 13:00:00 │
│ EWR    │  2013 │     1 │     1 │     9 │ 39.92  │ 28.04  │ 62.21  │ 260      │ 14.960139999999999 │ NA        │     0.0 │ 1012.7   │    10.0 │ 2013-01-01 14:00:00 │
│ EWR    │  2013 │     1 │     1 │    10 │ 41     │ 28.04  │ 59.65  │ 260      │ 13.809359999999998 │ NA        │     0.0 │ 1012.4   │    10.0 │ 2013-01-01 15:00:00 │
│ …      │     … │     … │     … │     … │ …      │ …      │ …      │ …        │ …                  │ …         │       … │ …        │       … │ …                   │
└────────┴───────┴───────┴───────┴───────┴────────┴────────┴────────┴──────────┴────────────────────┴───────────┴─────────┴──────────┴─────────┴─────────────────────┘

# The New York City flight data

Let’s use the [nycflights13 data](https://github.com/hadley/nycflights13) to predict whether a plane arrives more than 30 minutes late. This data set contains information on 325,819 flights departing near New York City in 2013. Let’s start by loading the data and making a few changes to the variables:

In [6]:
flight_data = (
    flights.mutate(
        # Convert the arrival delay to a factor
        # By default, PyTorch expects the target to have a Long datatype
        arr_delay=ibis.ifelse(flights.arr_delay >= 30, 1, 0).cast("int64"),
        # We will use the date (not date-time) in the recipe below
        date=flights.time_hour.date(),
    )
    # Include the weather data
    .inner_join(weather, ["origin", "time_hour"])
    # Only retain the specific columns we will use
    .select(
        "dep_time",
        "flight",
        "origin",
        "dest",
        "air_time",
        "distance",
        "carrier",
        "date",
        "arr_delay",
        "time_hour",
    )
    # Exclude missing data
    .dropna()
)
flight_data

┏━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ dep_time ┃ flight ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ carrier ┃ date       ┃ arr_delay ┃ time_hour           ┃
┡━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ time     │ int64  │ string │ string │ int64    │ int64    │ string  │ date       │ int64     │ timestamp(6)        │
├──────────┼────────┼────────┼────────┼──────────┼──────────┼─────────┼────────────┼───────────┼─────────────────────┤
│ 05:17:00 │   1545 │ EWR    │ IAH    │      227 │     1400 │ UA      │ 2013-01-01 │         0 │ 2013-01-01 10:00:00 │
│ 05:54:00 │    461 │ LGA    │ ATL    │      116 │      762 │ DL      │ 2013-01-01 │         0 │ 2013-01-01 11:00:00 │
│ 05:54:00 │   1696 │ EWR    │ ORD    │      150 │      719 │ UA      │ 2013-01-01 │         0 │ 2013-01-01 10:00:00 │
│ 05:55:00 │    507 │ EWR    │ FLL    │      158 │     1065 │ B6      │ 2013-01-01 │         0 │ 2013-01-01 11:00:00 │
│ 05:57:00 │   5708 │ LGA    │ IAD    │       53 │      229 │ EV      │ 2013-01-01 │         0 │ 2013-01-01 11:00:00 │
│ 05:57:00 │     79 │ JFK    │ MCO    │      140 │      944 │ B6      │ 2013-01-01 │         0 │ 2013-01-01 11:00:00 │
│ 05:58:00 │    301 │ LGA    │ ORD    │      138 │      733 │ AA      │ 2013-01-01 │         0 │ 2013-01-01 11:00:00 │
│ 05:58:00 │     49 │ JFK    │ PBI    │      149 │     1028 │ B6      │ 2013-01-01 │         0 │ 2013-01-01 11:00:00 │
│ 05:58:00 │     71 │ JFK    │ TPA    │      158 │     1005 │ B6      │ 2013-01-01 │         0 │ 2013-01-01 11:00:00 │
│ 05:58:00 │    194 │ JFK    │ LAX    │      345 │     2475 │ UA      │ 2013-01-01 │         0 │ 2013-01-01 11:00:00 │
│ …        │      … │ …      │ …      │        … │        … │ …       │ …          │         … │ …                   │
└──────────┴────────┴────────┴────────┴──────────┴──────────┴─────────┴────────────┴───────────┴─────────────────────┘

We can see that about 16% of the flights in this data set arrived more than 30 minutes late.

In [7]:
flight_data.arr_delay.value_counts().rename(n="arr_delay_count").mutate(
    prop=ibis._.n / ibis._.n.sum()
)

┏━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ arr_delay ┃ n      ┃ prop     ┃
┡━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ int64     │ int64  │ float64  │
├───────────┼────────┼──────────┤
│         0 │ 273279 │ 0.838745 │
│         1 │  52540 │ 0.161255 │
└───────────┴────────┴──────────┘

# Data splitting

To get started, let’s split this single dataset into two: a training set and a testing set. We’ll keep most of the rows in the original dataset (subset chosen randomly) in the training set. The training data will be used to fit the model, and the testing set will be used to measure model performance.

Because the order of rows in an Ibis table is undefined, we need a unique key to split the data reproducibly. [It is permissible for airlines to use the same flight number for different routes, as long as the flights do not operate on the same day. This means that the combination of the flight number and the date of travel is always unique.](https://www.euclaim.com/blog/flight-numbers-explained#:~:text=Can%20flight%20numbers%20be%20reused,of%20travel%20is%20always%20unique.)

In [8]:
flight_data_with_unique_key = flight_data.mutate(
    unique_key=ibis.literal(",").join(
        [flight_data.carrier, flight_data.flight.cast(str), flight_data.date.cast(str)]
    )
)
flight_data_with_unique_key

┏━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ dep_time ┃ flight ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ carrier ┃ date       ┃ arr_delay ┃ time_hour           ┃ unique_key         ┃
┡━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ time     │ int64  │ string │ string │ int64    │ int64    │ string  │ date       │ int64     │ timestamp(6)        │ string             │
├──────────┼────────┼────────┼────────┼──────────┼──────────┼─────────┼────────────┼───────────┼─────────────────────┼────────────────────┤
│ 05:57:00 │    461 │ LGA    │ ATL    │      100 │      762 │ DL      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ DL,461,2013-06-26  │
│ 05:58:00 │   4424 │ EWR    │ RDU    │       63 │      416 │ EV      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ EV,4424,2013-06-26 │
│ 05:58:00 │   6177 │ EWR    │ IAD    │       45 │      212 │ EV      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ EV,6177,2013-06-26 │
│ 06:00:00 │    731 │ LGA    │ DTW    │       78 │      502 │ DL      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ DL,731,2013-06-26  │
│ 06:01:00 │    684 │ EWR    │ LAX    │      316 │     2454 │ UA      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ UA,684,2013-06-26  │
│ 06:01:00 │    301 │ LGA    │ ORD    │      164 │      733 │ AA      │ 2013-06-26 │         1 │ 2013-06-26 10:00:00 │ AA,301,2013-06-26  │
│ 06:01:00 │   1837 │ LGA    │ MIA    │      148 │     1096 │ AA      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ AA,1837,2013-06-26 │
│ 06:01:00 │   1279 │ LGA    │ MEM    │      128 │      963 │ DL      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ DL,1279,2013-06-26 │
│ 06:02:00 │   1691 │ JFK    │ LAX    │      309 │     2475 │ UA      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ UA,1691,2013-06-26 │
│ 06:04:00 │   1447 │ JFK    │ CLT    │       75 │      541 │ US      │ 2013-06-26 │         0 │ 2013-06-26 10:00:00 │ US,1447,2013-06-26 │
│ …        │      … │ …      │ …      │        … │        … │ …       │ …          │         … │ …                   │ …                  │
└──────────┴────────┴────────┴────────┴──────────┴──────────┴─────────┴────────────┴───────────┴─────────────────────┴────────────────────┘

In [9]:
# FIXME(deepyaman): Proposed key isn't unique for actual departure date.
flight_data_with_unique_key.group_by("unique_key").mutate(
    cnt=flight_data_with_unique_key.count()
)[ibis._.cnt > 1]

┏━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ dep_time ┃ flight ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ carrier ┃ date       ┃ arr_delay ┃ time_hour           ┃ unique_key         ┃ cnt   ┃
┡━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ time     │ int64  │ string │ string │ int64    │ int64    │ string  │ date       │ int64     │ timestamp(6)        │ string             │ int64 │
├──────────┼────────┼────────┼────────┼──────────┼──────────┼─────────┼────────────┼───────────┼─────────────────────┼────────────────────┼───────┤
│ 19:59:00 │   1022 │ EWR    │ IAH    │      167 │     1400 │ UA      │ 2013-09-14 │         0 │ 2013-09-14 23:00:00 │ UA,1022,2013-09-14 │     2 │
│ 20:00:00 │   1022 │ EWR    │ IAH    │      186 │     1400 │ UA      │ 2013-09-14 │         0 │ 2013-09-14 00:00:00 │ UA,1022,2013-09-14 │     2 │
│ 19:12:00 │   1023 │ LGA    │ ORD    │      112 │      733 │ UA      │ 2013-05-29 │         0 │ 2013-05-29 23:00:00 │ UA,1023,2013-05-29 │     2 │
│ 21:16:00 │   1023 │ EWR    │ IAH    │      175 │     1400 │ UA      │ 2013-05-29 │         0 │ 2013-05-29 01:00:00 │ UA,1023,2013-05-29 │     2 │
│ 21:22:00 │   1052 │ EWR    │ IAH    │      173 │     1400 │ UA      │ 2013-08-27 │         0 │ 2013-08-27 01:00:00 │ UA,1052,2013-08-27 │     2 │
│ 15:18:00 │   1052 │ EWR    │ IAH    │      174 │     1400 │ UA      │ 2013-08-27 │         0 │ 2013-08-27 19:00:00 │ UA,1052,2013-08-27 │     2 │
│ 18:39:00 │   1053 │ EWR    │ CLE    │       72 │      404 │ UA      │ 2013-12-20 │         0 │ 2013-12-20 23:00:00 │ UA,1053,2013-12-20 │     2 │
│ 19:27:00 │   1053 │ EWR    │ CLE    │       69 │      404 │ UA      │ 2013-12-20 │         0 │ 2013-12-20 00:00:00 │ UA,1053,2013-12-20 │     2 │
│ 20:16:00 │   1071 │ EWR    │ BQN    │      196 │     1585 │ UA      │ 2013-02-26 │         0 │ 2013-02-26 01:00:00 │ UA,1071,2013-02-26 │     2 │
│ 17:20:00 │   1071 │ EWR    │ PHX    │      281 │     2133 │ UA      │ 2013-02-26 │         0 │ 2013-02-26 22:00:00 │ UA,1071,2013-02-26 │     2 │
│ …        │      … │ …      │ …      │        … │        … │ …       │ …          │         … │ …                   │ …                  │     … │
└──────────┴────────┴────────┴────────┴──────────┴──────────┴─────────┴────────────┴───────────┴─────────────────────┴────────────────────┴───────┘

In [10]:
import random

# Fix the random numbers by setting the seed
# This enables the analysis to be reproducible when random numbers are used
random.seed(222)

# Put 3/4 of the data into the training set
random_key = str(random.getrandbits(256))
data_split = flight_data_with_unique_key.mutate(
    train=(flight_data_with_unique_key.unique_key + random_key).hash().abs() % 4 < 3
)

# Create data frames for the two sets:
train_data = data_split[data_split.train].drop("unique_key", "train")
test_data = data_split[~data_split.train].drop("unique_key", "train")

# Create features

In [11]:
import ibis_ml as ml

flights_rec = ml.Recipe(
    ml.ExpandDate("date", components=["dow", "month"]),
    ml.Drop("date"),
    ml.TargetEncode(ml.nominal()),
    ml.DropZeroVariance(ml.everything()),
    ml.MutateAt("dep_time", ibis._.hour() * 60 + ibis._.minute()),
    ml.MutateAt(ml.timestamp(), ibis._.epoch_seconds()),
    # By default, PyTorch requires that the type of `X` is `np.float32`.
    # https://discuss.pytorch.org/t/mat1-and-mat2-must-have-the-same-dtype-but-got-double-and-float/197555/2
    ml.Cast(ml.numeric(), "float32"),
)

# Fit a model with a recipe

Let’s model the flight data. We can use any scikit-learn-compatible estimator.

In [12]:
import ipywidgets as widgets

w = widgets.RadioButtons(
    options=["scikit-learn", "XGBoost", "skorch (PyTorch)"],
    value="skorch (PyTorch)",  # Defaults to "skorch (PyTorch)"
    description="Library:",
)
w

RadioButtons(description='Library:', index=2, options=('scikit-learn', 'XGBoost', 'skorch (PyTorch)'), value='…

We will want to use our recipe across several steps as we train and test our model. We will:

1. **Process the recipe using the training set:** This involves any estimation or calculations based on the training set. For our recipe, the training set will be used to determine which predictors should be converted to dummy variables and which predictors will have zero-variance in the training set, and should be slated for removal.

1. **Apply the recipe to the training set:** We create the final predictor set on the training set.

1. **Apply the recipe to the test set:** We create the final predictor set on the test set. Nothing is recomputed and no information from the test set is used here; the dummy variable and zero-variance results from the training set are applied to the test set.

To simplify this process, we can use a [scikit-learn `Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

In [13]:
from sklearn.pipeline import Pipeline

if w.value == "scikit-learn":
    from sklearn.linear_model import LogisticRegression

    mod = LogisticRegression()
elif w.value == "XGBoost":
    import xgboost as xgb

    mod = xgb.XGBClassifier()
elif w.value == "skorch (PyTorch)":
    from torch import nn
    from skorch import NeuralNetClassifier

    class MyModule(nn.Module):
        def __init__(self, num_units=10, nonlin=nn.ReLU()):
            super().__init__()

            self.dense0 = nn.Linear(10, num_units)
            self.nonlin = nonlin
            self.dropout = nn.Dropout(0.5)
            self.dense1 = nn.Linear(num_units, num_units)
            self.output = nn.Linear(num_units, 2)
            self.softmax = nn.Softmax(dim=-1)

        def forward(self, X, **kwargs):
            X = self.nonlin(self.dense0(X))
            X = self.dropout(X)
            X = self.nonlin(self.dense1(X))
            X = self.softmax(self.output(X))
            return X

    mod = NeuralNetClassifier(
        MyModule,
        max_epochs=10,
        lr=0.1,
        # Shuffle training data on each epoch
        iterator_train__shuffle=True,
    )

pipe = Pipeline([("flights_rec", flights_rec), ("mod", mod)])

Now, there is a single function that can be used to prepare the recipe and train the model from the resulting predictors:

In [14]:
X_train = train_data.drop("arr_delay")
y_train = train_data.arr_delay
pipe.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.3911       0.8388        2.5698  1.1001
      2        4.9956       0.8388        2.5698  1.0937
      3        5.0020       0.8388        2.5698  1.0302
      4        4.9828       0.8388        2.5698  0.9660
      5        5.0307       0.8388        2.5698  1.0476
      6        5.0018       0.8388        2.5698  1.0403
      7        4.9952       0.8388        2.5698  0.9626
      8        4.9869       0.8388        2.5698  0.9641
      9        5.0159       0.8388        2.5698  0.9952
     10        5.0151       0.8388        2.5698  0.9532


Pipeline(steps=[('flights_rec', <ibis_ml.core.Recipe object at 0x149360730>),
                ('mod',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense0): Linear(in_features=10, out_features=10, bias=True)
    (nonlin): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
    (softmax): Softmax(dim=-1)
  ),
))])

# Use a trained workflow to predict

...

In [15]:
X_test = test_data.drop("arr_delay")
y_test = test_data.arr_delay
pipe.score(X_test, y_test)

0.8385534190130481